# Data Mining Lab 2
#### Team 2:  Patricia Goresen, Jeffrey Lancon, Brychan Manry, George Sturrock

## Introduction
The source of baseball data for this lab is the Sean Lahman Baseball Database [ http://www.seanlahman.com/baseball-archive/statistics/]. Often cited as the most complete baseball database, the data set includes twenty-seven data tables and millions of records covering most non-proprietary baseball data pertaining to offense, fielding, pitching, payroll, player demographics, team statistics, manager data and much more. In past assignments, this team has focused on team level data.  For this Lab, the team will focus on player level data.  In particular, the focus will be on position players.  Pitchers will not be viewed in this study.  This will allow for more exploration of baseball data and provide a broader variety of regression and classification scenarios.  

This lab will focus on the analysis of four specific tables containing player level details in the Lahman Baseball Database:  Batting, Fielding, Player Awards and Salaries.  The names of the different tables accurately describe the contents.  For example, the Batting table contains offensive statics per year for each player.  The tables are at different grains.  The utlimate intention is to have a single dataframe accurately combining all relevant features of the four tables.  The methods used to achieve this goal will be presented below.  

In this Lab, the team will focus on two predictive tasks.  First, models will be trained to predict the salary of a player.  Second, a classification models will be created to attempt to categorize players by position using attributes other than position.  Three different models will be created for each predictive task.

## Data Preparation - Part 1
### Import Data

In [1]:
import pandas as pd
import numpy as np

awards = pd.read_csv('~/7331_Lab2/data/AwardsPlayers.csv')
batting = pd.read_csv('~/7331_Lab2/data/Batting.csv')
fielding = pd.read_csv('~/7331_Lab2/data/Fielding.csv')
salaries = pd.read_csv('~/7331_Lab2/data/Salaries.csv')

print('Awards', awards.info())
print('Batting', batting.info())
print('Fielding', fielding.info())
print('Salaries', salaries.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6158 entries, 0 to 6157
Data columns (total 6 columns):
playerID    6158 non-null object
awardID     6158 non-null object
yearID      6158 non-null int64
lgID        6158 non-null object
tie         47 non-null object
notes       4710 non-null object
dtypes: int64(1), object(5)
memory usage: 288.7+ KB
Awards None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104324 entries, 0 to 104323
Data columns (total 24 columns):
playerID     104324 non-null object
yearID       104324 non-null int64
stint        104324 non-null int64
teamID       104324 non-null object
lgID         103586 non-null object
G            104324 non-null int64
G_batting    0 non-null float64
AB           104324 non-null int64
R            104324 non-null int64
H            104324 non-null int64
2B           104324 non-null int64
3B           104324 non-null int64
HR           104324 non-null int64
RBI          103568 non-null float64
SB           101956 non-null flo

In [2]:
## Go back and convert to table ##
#teamNullCols = pd.DataFrame({'n Missing' : teamNullCols, '% Missing' : ((teamNullCols / teams2.shape[0])*100).round(2)#.astype(str) + "%"})

print("Shape of DataFrames:", "Awards:", awards.shape, "Batting:", batting.shape, "Fielding:", fielding.shape, "Salaries:", salaries.shape)

Shape of DataFrames: Awards: (6158, 6) Batting: (104324, 24) Fielding: (138838, 18) Salaries: (26428, 5)


### Subsetting by Year

As can be seen above, a one to one match does not exist between the four source tables.  The process of creating dataframes which can be merged into a single dataframe will begin with row level filtering.  Only records from the 1985 season and greater will be used for analysis in this assignment.  1985 was chosen for two reasons.  First, salary data only goes back to 1985.  Second, the evolution of the game and it players has changed quite a bit since beginning of the professional game of baseball.  Selecting only records from the 1985 season and forward providees a 32 year data set while eliminating data that would likely lead to erroneous results.  

In [3]:
awards = awards[awards.yearID > 1984]
batting = batting[batting.yearID > 1984]
fielding = fielding[fielding.yearID > 1984]
salaries = salaries[salaries.yearID > 1984]

print("Shape of DataFrames:", "Awards:", awards.shape, "Batting:", batting.shape, "Fielding:", fielding.shape, "Salaries:", salaries.shape)

Shape of DataFrames: Awards: (2398, 6) Batting: (42071, 24) Fielding: (54560, 18) Salaries: (26428, 5)


### Batting

The Batting table will be the driver for the overall consolidated data frame.  In other words, fielding, salaries and awards will be merged into the Batting data frame.  First, categorical variables such as Team and League ID which offer limited predictive or structural value will be dropped from the data frame.  The G_old column is missing values in every row and will be dropped as well.

In [4]:
batting = batting.drop(['teamID', 'lgID', 'G_old', 'G_batting', 'stint'], axis=1)
#Drop rows with no basic statistics are recorded in a year
batting = batting[(batting[['AB', 'BB']] != 0).all(axis=1)]
print(batting.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22527 entries, 53 to 104305
Data columns (total 19 columns):
playerID    22527 non-null object
yearID      22527 non-null int64
G           22527 non-null int64
AB          22527 non-null int64
R           22527 non-null int64
H           22527 non-null int64
2B          22527 non-null int64
3B          22527 non-null int64
HR          22527 non-null int64
RBI         22527 non-null float64
SB          22527 non-null float64
CS          22527 non-null float64
BB          22527 non-null int64
SO          22527 non-null float64
IBB         22527 non-null float64
HBP         22527 non-null float64
SH          22527 non-null float64
SF          22527 non-null float64
GIDP        22527 non-null float64
dtypes: float64(9), int64(9), object(1)
memory usage: 3.4+ MB
None


In [5]:
#df = batting.groupby(['playerID', 'yearID']).count().reset_index()
#print(df)

There are approximately 1300 instances remaining where a player has more than one record per year in the batting data frame.  This is to be expected as player trades and multiple stints in the major league do occurr in a single season.  These instances will be consolidated into a single row.

In [6]:
#Identify Instances where a player has more than one batting record per year.
df = batting.groupby(['playerID', 'yearID']).count().reset_index()
df = df[df.G > 1]
print(df)

#Consolidate and Aggreagate
print(batting.groupby(['yearID', 'playerID']).sum().reset_index())
battingFinal = batting.groupby(['yearID', 'playerID']).sum().reset_index()

### Good Code Save for Future ###
#aggRules = {'stint':['count'], 'G':['sum'], 'AB':['sum'], 'R':['sum'], 'H':['sum'], '2B':['sum'], '3B':['sum'], 'HR':['sum'], \
#            'RBI':['sum'], 'SB':['sum'], 'CS':['sum'], 'BB':['sum'], 'SO':['sum'], 'IBB':['sum'], 'HBP':['sum'], 'SH':['sum'], \
#           'SF':['sum'],'GIDP':['sum']}
#
#battingFinal = batting.groupby(['playerID', 'yearID']).agg(aggRules).reset_index()

print(battingFinal)
print(battingFinal.info())
print(battingFinal.describe())

        playerID  yearID  G  AB  R  H  2B  3B  HR  RBI  SB  CS  BB  SO  IBB  \
11     abbotku01    1998  2   2  2  2   2   2   2    2   2   2   2   2    2   
26     abnersh01    1991  2   2  2  2   2   2   2    2   2   2   2   2    2   
38     abreubo01    2006  2   2  2  2   2   2   2    2   2   2   2   2    2   
44     abreubo01    2012  2   2  2  2   2   2   2    2   2   2   2   2    2   
67     ackledu01    2015  2   2  2  2   2   2   2    2   2   2   2   2    2   
80     adamsma01    2017  2   2  2  2   2   2   2    2   2   2   2   2    2   
107    agbaybe01    2002  2   2  2  2   2   2   2    2   2   2   2   2    2   
112    aguaylu01    1988  2   2  2  2   2   2   2    2   2   2   2   2    2   
141    aldremi01    1991  2   2  2  2   2   2   2    2   2   2   2   2    2   
145    aldremi01    1996  2   2  2  2   2   2   2    2   2   2   2   2    2   
148    alexama02    1997  2   2  2  2   2   2   2    2   2   2   2   2    2   
168    alfoned01    2006  2   2  2  2   2   2   2   

       yearID   playerID    G   AB    R    H  2B  3B  HR    RBI    SB    CS  \
0        1985  adamsri02   54  121   12   23   3   1   2   10.0   1.0   1.0   
1        1985  aguaylu01   91  165   27   46   7   3   6   21.0   1.0   0.0   
2        1985  aguilri01   22   36    1   10   2   0   0    2.0   0.0   0.0   
3        1985  aikenwi01   12   20    2    4   1   0   1    5.0   0.0   0.0   
4        1985  almonbi01   88  244   33   66  17   0   6   29.0  10.0   7.0   
5        1985  anderda02   77  221   24   44   6   0   4   18.0   5.0   4.0   
6        1985  andujjo01   38   94    2   10   2   0   0    8.0   3.0   1.0   
7        1985  armasto01  103  385   50  102  17   5  23   64.0   0.0   0.0   
8        1985  ashbyal01   65  189   20   53   8   0   8   25.0   0.0   0.0   
9        1985  ayalabe01   46   76   10   19   7   0   2   15.0   0.0   0.0   
10       1985  backmwa01  145  520   77  142  24   5   1   38.0  30.0  12.0   
11       1985  bailema01  114  332   47   88  14   0

max       39.000000     17.000000     35.000000  


In [7]:
print("Index:", battingFinal.index)
print("Columns:", battingFinal.columns)

Index: RangeIndex(start=0, stop=21160, step=1)
Columns: Index(['yearID', 'playerID', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
       'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP'],
      dtype='object')


### Fielding
The Fielding table contains most of the common defensive statistics in baseball.  The grain of this table is by player, year, sting and position.  It is common for players to play more than one position over the course of a season (year).  However, most players have a primary position.  There is also the concept of a utlity player.  This a player who has no primary position and plays a significant number of games in many positions.  The end objective is to produce a final fielding file with data summarized at the player and year grain.  As this lab is focused on everyday position players, pitchers will be dropped from the fielding data frame.  As this lab is focused on everyday position players, pitchers will be dropped from the fielding data frame.  

In [8]:
fielding = fielding[fielding.POS != 'P']
print(fielding.info())
print(fielding.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33738 entries, 76 to 138819
Data columns (total 18 columns):
playerID    33738 non-null object
yearID      33738 non-null int64
stint       33738 non-null int64
teamID      33738 non-null object
lgID        33738 non-null object
POS         33738 non-null object
G           33738 non-null int64
GS          33727 non-null float64
InnOuts     33727 non-null float64
PO          33738 non-null int64
A           33738 non-null int64
E           33738 non-null float64
DP          33738 non-null int64
PB          3651 non-null float64
WP          0 non-null float64
SB          3651 non-null float64
CS          3651 non-null float64
ZR          0 non-null float64
dtypes: float64(8), int64(6), object(4)
memory usage: 4.9+ MB
None
             yearID         stint             G            GS       InnOuts  \
count  33738.000000  33738.000000  33738.000000  33727.000000  33727.000000   
mean    2001.689697      1.077153     41.294297     35.961781

From the above describe statement, it's easy to see WP (wild pitch) and ZR (zone rating) are missing values in the entire column.  The two columns will be dropped from the fielding data frame.  To maintain consistency with the batting data frame, team and league will be dropped as well.  

In [9]:
fielding = fielding.drop(['teamID', 'lgID', 'WP', 'ZR'], axis=1)

There is high rate of missing values in the "PB", "SB" and "CS" columns.  This does make sense as these are statistics specific to catchers.  The data was spot checked to verify these are accurate missing values.  These missing values will be replaced with zeroes.

In [10]:
fielding.fillna(value={'PB': 0, 'SB': 0, 'ZR': 0, 'CS': 0}, inplace = True)
print(fielding.info())
print(fielding.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33738 entries, 76 to 138819
Data columns (total 14 columns):
playerID    33738 non-null object
yearID      33738 non-null int64
stint       33738 non-null int64
POS         33738 non-null object
G           33738 non-null int64
GS          33727 non-null float64
InnOuts     33727 non-null float64
PO          33738 non-null int64
A           33738 non-null int64
E           33738 non-null float64
DP          33738 non-null int64
PB          33738 non-null float64
SB          33738 non-null float64
CS          33738 non-null float64
dtypes: float64(6), int64(6), object(2)
memory usage: 3.9+ MB
None
             yearID         stint             G            GS       InnOuts  \
count  33738.000000  33738.000000  33738.000000  33727.000000  33727.000000   
mean    2001.689697      1.077153     41.294297     35.961781    963.608830   
std        9.454190      0.282277     47.472854     46.329978   1218.781043   
min     1985.000000      1.000

In [11]:
#Identify Players who played multiple positions per year.
df = fielding.groupby(['playerID', 'yearID']).count().reset_index()
df = df[df.stint > 1]
print(df)

        playerID  yearID  stint  POS  G  GS  InnOuts  PO  A  E  DP  PB  SB  CS
1       abadan01    2003      2    2  2   2        2   2  2  2   2   2   2   2
7      abbotku01    1993      3    3  3   3        3   3  3  3   3   3   3   3
10     abbotku01    1996      3    3  3   3        3   3  3  3   3   3   3   3
11     abbotku01    1997      4    4  4   4        4   4  4  4   4   4   4   4
12     abbotku01    1998      7    7  7   7        7   7  7  7   7   7   7   7
13     abbotku01    1999      4    4  4   4        4   4  4  4   4   4   4   4
14     abbotku01    2000      4    4  4   4        4   4  4  4   4   4   4   4
15     abbotku01    2001      2    2  2   2        2   2  2  2   2   2   2   2
21     abernbr01    2003      2    2  2   2        2   2  2  2   2   2   2   2
22     abernbr01    2005      2    2  2   2        2   2  2  2   2   2   2   2
27     abnersh01    1991      2    2  2   2        2   2  2  2   2   2   2   2
39     abreubo01    2006      2    2  2   2        2

In [12]:
#Identify Primary Postion per Player per Year Based on Max Games at a position.
df = fielding.groupby(['playerID', 'yearID', 'POS']).sum().reset_index()
df = df.groupby(['playerID', 'yearID', 'POS'])['G'].max().reset_index()
#df = df.groupby(['playerID', 'yearID', 'POS']).max()['G']
df = df.sort_values(by=['playerID', 'yearID', 'G'], ascending=[True, True, False])
print(df)
print(df.describe())

primePOS = pd.DataFrame()
svPlayerID = ''
svYearID = 0

for index, row in df.iterrows():
    #print (row["playerID"], row["yearID"], row['POS'], row['G'])
    if (svPlayerID != row['playerID'] or svYearID != row['yearID']):
        primePOS = primePOS.append({'playerID': row['playerID'], 'yearID': row['yearID'], 'Primary': row['POS']}, ignore_index=True)
        #print(svPlayerID, row["playerID"], svYearID, row["yearID"], row['POS'], row['G'])
        svPlayerID = row['playerID']
        svYearID = row['yearID']
    else:
        continue
  

        playerID  yearID POS    G
0       abadan01    2001  1B    1
1       abadan01    2003  1B    7
2       abadan01    2003  OF    1
3      abbotje01    1997  OF   10
4      abbotje01    1998  OF   76
5      abbotje01    1999  OF   17
6      abbotje01    2000  OF   65
7      abbotje01    2001  OF   17
9      abbotku01    1993  OF   13
10     abbotku01    1993  SS    6
8      abbotku01    1993  2B    2
11     abbotku01    1994  SS   99
12     abbotku01    1995  SS  115
15     abbotku01    1996  SS   44
14     abbotku01    1996  3B   33
13     abbotku01    1996  2B   20
16     abbotku01    1997  2B   54
18     abbotku01    1997  OF   10
19     abbotku01    1997  SS    7
17     abbotku01    1997  3B    4
23     abbotku01    1998  SS   35
22     abbotku01    1998  OF   14
20     abbotku01    1998  2B    7
21     abbotku01    1998  3B    4
25     abbotku01    1999  2B   66
24     abbotku01    1999  1B    8
26     abbotku01    1999  OF    4
27     abbotku01    1999  SS    3
31     abbotku

In [13]:
print(primePOS)

      Primary   playerID  yearID
0          1B   abadan01  2001.0
1          1B   abadan01  2003.0
2          OF  abbotje01  1997.0
3          OF  abbotje01  1998.0
4          OF  abbotje01  1999.0
5          OF  abbotje01  2000.0
6          OF  abbotje01  2001.0
7          OF  abbotku01  1993.0
8          SS  abbotku01  1994.0
9          SS  abbotku01  1995.0
10         SS  abbotku01  1996.0
11         2B  abbotku01  1997.0
12         SS  abbotku01  1998.0
13         2B  abbotku01  1999.0
14         SS  abbotku01  2000.0
15         2B  abbotku01  2001.0
16         OF  abercre01  2006.0
17         OF  abercre01  2007.0
18         OF  abercre01  2008.0
19         2B  abernbr01  2001.0
20         2B  abernbr01  2002.0
21         2B  abernbr01  2003.0
22         2B  abernbr01  2005.0
23         OF  abnersh01  1987.0
24         OF  abnersh01  1988.0
25         OF  abnersh01  1989.0
26         OF  abnersh01  1990.0
27         OF  abnersh01  1991.0
28         OF  abnersh01  1992.0
29        

In [14]:
#Condense Fielding Dataframe into one row per player per year.
fielding = fielding.groupby(['playerID', 'yearID']).sum().reset_index()

#Merge Primary Position into condensed Fielding data frame
fieldingFinal = pd.merge(fielding, primePOS, on=['playerID', 'yearID'], how='left')

print(fieldingFinal)
print(fieldingFinal.describe())

        playerID yearID  stint    G     GS  InnOuts    PO    A     E  DP  \
0       abadan01   2001      1    1    0.0      3.0     2    0   0.0   1   
1       abadan01   2003      2    8    4.0    135.0    35    1   1.0   2   
2      abbotje01   1997      1   10    4.0    140.0    15    0   0.0   0   
3      abbotje01   1998      1   76   61.0   1526.0   132    0   4.0   0   
4      abbotje01   1999      1   17   17.0    408.0    25    0   1.0   0   
5      abbotje01   2000      1   65   50.0   1357.0   101    2   2.0   0   
6      abbotje01   2001      1   17    8.0    257.0    26    0   1.0   0   
7      abbotku01   1993      3   21   14.0    414.0    36   13   2.0   2   
8      abbotku01   1994      1   99   93.0   2476.0   162  260  15.0  61   
9      abbotku01   1995      1  115  109.0   2925.0   149  290  19.0  66   
10     abbotku01   1996      3   97   83.0   2156.0   123  205  12.0  43   
11     abbotku01   1997      4   75   57.0   1491.0   126  136   8.0  27   
12     abbot

### Salaries

The Salaries table contains yearly salary figures for players beginning in 1985.  This data will be condensed into one row per player per year similar to the fielding and batting tables.  League and Team will be dropped from this data frame as well.

In [15]:
#Drop League and Team
salaries = salaries.drop(['teamID', 'lgID'], axis=1)

#Summarize salaries at the year and player level
salariesFinal = salaries.groupby(['playerID', 'yearID']).sum().reset_index()

print(salariesFinal)
print(salariesFinal.describe())

        playerID  yearID    salary
0      aardsda01    2004    300000
1      aardsda01    2007    387500
2      aardsda01    2008    403250
3      aardsda01    2009    419000
4      aardsda01    2010   2750000
5      aardsda01    2011   4500000
6      aardsda01    2012    500000
7       aasedo01    1986    600000
8       aasedo01    1987    625000
9       aasedo01    1988    675000
10      aasedo01    1989    400000
11      abadan01    2006    327000
12      abadfe01    2011    418000
13      abadfe01    2012    485000
14      abadfe01    2014    525900
15      abadfe01    2015   1087500
16      abadfe01    2016   1250000
17     abbotje01    1998    175000
18     abbotje01    1999    255000
19     abbotje01    2000    255000
20     abbotje01    2001    300000
21     abbotji01    1989     68000
22     abbotji01    1990    185000
23     abbotji01    1991    357500
24     abbotji01    1992   1850000
25     abbotji01    1993   2350000
26     abbotji01    1994   2775000
27     abbotji01    

### Merge Batting, Fielding and Salaries

In [16]:
#Merge Batting and Fielding
#print(pd.merge(battingFinal, fieldingFinal, on=['playerID', 'yearID'], how='left'))
player = pd.merge(battingFinal, fieldingFinal, on=['playerID', 'yearID'], how='left')

#Merge Salaries
player = pd.merge(player, salariesFinal, on=['playerID', 'yearID'], how='left')

print(player)
print(player.info())
print(player.describe())

      yearID   playerID  G_x   AB    R    H  2B  3B  HR    RBI    ...      \
0       1985  adamsri02   54  121   12   23   3   1   2   10.0    ...       
1       1985  aguaylu01   91  165   27   46   7   3   6   21.0    ...       
2       1985  aguilri01   22   36    1   10   2   0   0    2.0    ...       
3       1985  aikenwi01   12   20    2    4   1   0   1    5.0    ...       
4       1985  almonbi01   88  244   33   66  17   0   6   29.0    ...       
5       1985  anderda02   77  221   24   44   6   0   4   18.0    ...       
6       1985  andujjo01   38   94    2   10   2   0   0    8.0    ...       
7       1985  armasto01  103  385   50  102  17   5  23   64.0    ...       
8       1985  ashbyal01   65  189   20   53   8   0   8   25.0    ...       
9       1985  ayalabe01   46   76   10   19   7   0   2   15.0    ...       
10      1985  backmwa01  145  520   77  142  24   5   1   38.0    ...       
11      1985  bailema01  114  332   47   88  14   0  10   45.0    ...       

                G_x            AB             R             H            2B  \
count  21160.000000  21160.000000  21160.000000  21160.000000  21160.000000   
mean      78.567250    241.614698     32.513658     63.500709     12.251843   
std       49.956058    199.172162     31.295678     57.532276     11.873773   
min        1.000000      1.000000      0.000000      0.000000      0.000000   
25%       32.000000     58.000000      5.000000     11.000000      2.000000   
50%       76.000000    190.000000     22.000000     46.000000      8.000000   
75%      125.000000    416.000000     54.000000    110.000000     20.000000   
max      163.000000    716.000000    152.000000    262.000000     59.000000   

                 3B            HR           RBI          SB_x          CS_x  \
count  21160.000000  21160.000000  21160.000000  21160.000000  21160.000000   
mean       1.372732      7.120227     30.824149      4.624433      1.981711   
std        2.090482      9.316208     31.238458    

### Missing Values

### Scaling

#### Write Player CSV file

In [17]:
player.to_csv('~/7331_Lab2/data/player.csv', sep=',', encoding='utf-8')